# 2.2 An AMPL model for the diet problem

Clearly we will have to consider more extensive modifications to our
linear program in order to produce a diet that is even remotely acceptable.
We will probably want to change the sets of food and nutrients, as well as
the nature of the constraints and bounds.  As in the
production example of the previous chapter, this will be much easier to do
if we rely on a general model that can be coupled with
a variety of specific data files.

This model deals with two things: nutrients
and foods.  Thus we begin an `AMPL` model by declaring sets of
each:
```
set NUTR;
set FOOD;
```
Next we need to specify the numbers required by the model.  Certainly a
positive cost should be given for each food:
```
param cost {FOOD} > 0;
```
We also specify that
for each food there are lower and upper limits on the number of packages
in the diet:
```
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];
```
Notice that we need a dummy index
`j`
to run over
`FOOD`
in the declaration of
`f_max` ,
in order to say that the maximum for each food must be greater than
or equal to the corresponding minimum.

To make this model
somewhat more general than our examples so far, we also specify similar lower and upper limits
on the amount of each nutrient in the diet:
```
param n_min {NUTR} >= 0;
param n_max {i in NUTR} >= n_min[i];
```
Finally, for each combination of a nutrient and a food, we need a number
that represents the amount of the nutrient in one package of the food.  You
may recall from [Chapter 1](../01/01.md) that such a "product" of two sets is
written by listing them both:
```
param amt {NUTR,FOOD} >= 0;
```
References to this parameter require two indices.  For example,
`amt[i,j]`
is
the amount of nutrient
`i`
in a package of food
`j` .

The decision variables for this model are the numbers of packages to
buy of the different foods:
```
var Buy {j in FOOD} >= f_min[j], <= f_max[j];
```
The number of packages of some food
`j`
to be bought will be called
`Buy[j]` ;
in any acceptable solution it will have to lie between
`f_min[j]`
and
`f_max[j]` .

The total cost of buying a food
`j`
is the cost per package,
`cost[j]` ,
times the number of packages,
`Buy[j]` .
The objective to be minimized is
the sum of this product over all foods
`j` :
```
minimize Total_Cost:  sum {j in FOOD} cost[j] * Buy[j];
```
This
`minimize`
declaration works the same as
`maximize`
did in Chapter 1.

Similarly, the amount of a nutrient
`i`
supplied by a food
`j`
is the nutrient per
package,
`amt[i,j]` ,
times the number of packages
`Buy[j]` .
The total amount of nutrient
`i`
supplied is the sum of this product over all foods
`j` :
```
sum {j in FOOD} amt[i,j] * Buy[j]
```
To complete the model, we need only specify that each
such sum must lie between the appropriate bounds.  Our
constraint declaration begins
```
subject to Diet {i in NUTR}:
```
to say that a constraint named
`Diet[i]`
must be imposed for each member
`i`
of
`NUTR` .
The rest of the declaration gives the algebraic statement of the
constraint for nutrient
`i` :
the variables must satisfy
```
n_min[i] <= sum {j in FOOD} amt[i,j] * Buy[j] <= n_max[i]
```
A "double inequality" like this is interpreted in the obvious way:  the
value of the sum in the middle must lie between
`n_min[i]`
and
`n_max[i]` .
The complete model is shown bellow.

<a id='fig-2-1'><center><b>Figure 2-1:</b> Diet model in AMPL (diet.mod).</center></a>

In [1]:
%%writefile diet.mod

set NUTR;
set FOOD;

param cost {FOOD} > 0;
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];

param n_min {NUTR} >= 0;
param n_max {i in NUTR} >= n_min[i];

param amt {NUTR,FOOD} >= 0;

var Buy {j in FOOD} >= f_min[j], <= f_max[j];

minimize Total_Cost:  sum {j in FOOD} cost[j] * Buy[j];

subject to Diet {i in NUTR}:
   n_min[i] <= sum {j in FOOD} amt[i,j] * Buy[j] <= n_max[i];

Overwriting diet.mod
